In [1]:
!pip install transformers


In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import transformers
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from transformers import DistilBertModel, DistilBertTokenizer
from sklearn import metrics
from tqdm import tqdm
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_data = pd.read_csv("/content/drive/MyDrive/airline_sentiment_analysis.csv").dropna()
train_data.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,airline_sentiment,text
0,1,positive,@VirginAmerica plus you've added commercials t...
1,3,negative,@VirginAmerica it's really aggressive to blast...
2,4,negative,@VirginAmerica and it's a really big bad thing...
3,5,negative,@VirginAmerica seriously would pay $30 a fligh...
4,6,positive,"@VirginAmerica yes, nearly every time I fly VX..."


In [4]:
import re
!pip install emoji
import emoji
def process_tweet(tweet):
    new_tweet = tweet.lower()
    new_tweet = re.sub(r'@\w+', '', new_tweet) # Remove @s
    new_tweet = re.sub(r'#', '', new_tweet) # Remove hashtags
    new_tweet = re.sub(r':', ' ', emoji.demojize(new_tweet)) # Turn emojis into words
    new_tweet = re.sub(r'http\S+', '',new_tweet) # Remove URLs
    new_tweet = re.sub(r'\$\S+', 'dollar', new_tweet) # Change dollar amounts to dollar
    new_tweet = re.sub(r'[^a-z0-9\s]', '', new_tweet) # Remove punctuation
    new_tweet = re.sub(r'[0-9]+', 'number', new_tweet) # Change number values to number
    return new_tweet

def binary(sentiment):
  if sentiment == "positive":
    return 1
  else:
    return 0 
train_data["text"] = train_data["text"].apply(process_tweet)
train_data["airline_sentiment"] = train_data["airline_sentiment"].apply(binary)


In [5]:
train_data.head()

,Unnamed: 0,airline_sentiment,text
0,1,1,plus youve added commercials to the experienc...
1,3,0,its really aggressive to blast obnoxious ente...
2,4,0,and its a really big bad thing about it
3,5,0,seriously would pay dollar a flight for seats...
4,6,1,yes nearly every time i fly vx this ear worm ...


In [6]:
train_data = train_data.drop(train_data.columns[0], axis=1)

In [7]:
#train_data , test_data = train_test_split(train_data,test_size=0.1)
#train_data.head()

In [8]:
#test_data.head()

In [9]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

model, pretrained_weights = (transformers.DistilBertModel, 'distilbert-base-uncased')
model = model.from_pretrained(pretrained_weights)

#for child in model.children():
#  for param in child.parameters():
#      param.requires_grad = False

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
print(' Original: ', train_data["text"][1])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(train_data["text"][1]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_data["text"][1])))

 Original:   its really aggressive to blast obnoxious entertainment in your guests faces amp they have little recourse
Tokenized:  ['its', 'really', 'aggressive', 'to', 'blast', 'ob', '##no', '##xious', 'entertainment', 'in', 'your', 'guests', 'faces', 'amp', 'they', 'have', 'little', 'rec', '##ours', '##e']
Token IDs:  [2049, 2428, 9376, 2000, 8479, 27885, 3630, 25171, 4024, 1999, 2115, 6368, 5344, 23713, 2027, 2031, 2210, 28667, 22957, 2063]


In [11]:
#x_token = x_train.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)[:300]))

#padded = np.array([i + [0]*(300 -len(i)) for i in x_token.values])

In [12]:
#attention_mask = np.where(padded != 0, 1, 0)


In [13]:
class tweet_Dataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        review = str(self.data["text"][index])
        inputs = self.tokenizer.encode_plus(
            review,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation = True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data["airline_sentiment"][index], dtype=torch.long)
        }
        
    def __len__(self):
        return len(self.data)
  
train_dataset = tweet_Dataset(train_data, tokenizer,100)

train_dataset , test_dataset = torch.utils.data.random_split(train_dataset, [10541, 1000])
print(len(train_dataset))
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,drop_last = True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True,drop_last = True)


10541


In [14]:
class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.distill_bert = model
        self.drop = torch.nn.Dropout(0.1)
        self.out = torch.nn.Linear(768, 2)
    
    def forward(self, ids, mask):
        distilbert_output = self.distill_bert(ids, mask)
        hidden_state = distilbert_output[0]  # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        output_1 = self.drop(pooled_output)
        output = self.out(output_1)
        return output
    
model = DistillBERTClass()
model.to(device)

DistillBERTClass(
  (distill_bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Line

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [16]:
def accuracy(y_pred, y):
    _, predicted = torch.max(torch.sigmoid(y_pred).data, 1)
    total = y.size(0)
    correct = (predicted == y).sum().item()
    return correct


In [17]:

for epoch in range(2):
    acc = 0
    for i,data in enumerate(train_loader,0):
        optimizer.zero_grad()
        input_ids = data['ids'].to(device)
        attention_mask = data['mask'].to(device)
        labels = data['targets'].to(device)
        outputs = model(input_ids,attention_mask)
        #loss = nn.functional.binary_cross_entropy(nn.functional.sigmoid(outputs), labels)
        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()
        acc += accuracy(outputs,labels)
        if i%50 == 49:
            print('[%d, %d] accuracy: %.3f' %
                  (epoch + 1, i + 1, acc / ((i+1)*16)))





/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[1, 50] accuracy: 0.770
[1, 100] accuracy: 0.801
[1, 150] accuracy: 0.798
[1, 200] accuracy: 0.805
[1, 250] accuracy: 0.814
[1, 300] accuracy: 0.829
[1, 350] accuracy: 0.839
[1, 400] accuracy: 0.849
[1, 450] accuracy: 0.857
[1, 500] accuracy: 0.863
[1, 550] accuracy: 0.868
[1, 600] accuracy: 0.872
[1, 650] accuracy: 0.876
[2, 50] accuracy: 0.917
[2, 100] accuracy: 0.924
[2, 150] accuracy: 0.923
[2, 200] accuracy: 0.924
[2, 250] accuracy: 0.926
[2, 300] accuracy: 0.929
[2, 350] accuracy: 0.928
[2, 400] accuracy: 0.930
[2, 450] accuracy: 0.929
[2, 500] accuracy: 0.932
[2, 550] accuracy: 0.931
[2, 600] accuracy: 0.932
[2, 650] accuracy: 0.932


In [18]:
torch.save(model, "weights.pth")

In [19]:

acc = 0
with torch.no_grad():
    for i,data in enumerate(test_loader,0):
        model.eval()
        input_ids = data['ids'].to(device)
        attention_mask = data['mask'].to(device)
        labels = data['targets'].to(device)
        outputs = model(input_ids,attention_mask)
        acc += accuracy(outputs,labels)
print(i)
acc/((i+1)*16)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


61


0.9425403225806451